In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Basic Imports


In [ ]:
pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 1.4 MB 68.6 MB/s 
     |████████████████████████████████| 65 kB 4.7 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17416 sha256=2f9a7bb26c37e5db669d84b1f39a13797e5bbb9317489a9be5b4b41cb75d2b21
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [ ]:
import os, sys
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from googletrans import Translator
import numpy as np
import matplotlib.pyplot as plt

### Constants

In [ ]:
BATCH_SIZE = 64
EPOCHS = 20
LSTM_NODES =256
NUM_SENTENCES = 20000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

### Loading data

In [ ]:
input_texts = [] # sentence in original language
target_texts = [] # sentence in target language
target_texts_inputs = [] # sentence in target language offset by 1

In [ ]:
text = 0

for line in open('./drive/MyDrive/Speech Conversion/ben2.txt',encoding='utf8'):
    
    text = text+1
    
    if text > NUM_SENTENCES:
        break
   
    if '\t' not in line:
        continue
        
    input_text, translation = line.split('\t')
    
    target_text = translation + ' <eos>'
    target_text_input = '<sos> ' + translation
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_inputs.append(target_text_input)
    
print('Number of Samples:', len(input_texts))

Number of Samples: 4619


In [ ]:
print(len(target_texts))

4619


In [ ]:
print(len(target_texts_inputs))

4619


#### Tokenizer 1 (inputs)

In [ ]:
tokenizer_inputs = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences = tokenizer_inputs.texts_to_sequences(input_texts)

#### Word to index mapping (inputs)

In [ ]:
word2idx_inputs = tokenizer_inputs.word_index
print('Found {} unique tokens:'.format(len(word2idx_inputs)))

Found 1902 unique tokens:


In [ ]:
max_len_input = max(len(s) for s in input_sequences)
print('Max input length',max_len_input)

Max input length 20


#### Tokenizer 2 (targets)

In [ ]:
tokenizer_outputs = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
tokenizer_outputs.fit_on_texts(target_texts + target_texts_inputs) 
target_sequences = tokenizer_outputs.texts_to_sequences(target_texts)
target_sequences_inputs = tokenizer_outputs.texts_to_sequences(target_texts_inputs)

#### Word to index mapping (targets)

In [ ]:
word2idx_outputs = tokenizer_outputs.word_index
print('Found {} unique tokens:'.format(len(word2idx_outputs)))

Found 3657 unique tokens:


In [ ]:
num_words_output = len(word2idx_outputs) + 1

In [ ]:
max_len_target = max(len(s) for s in target_sequences)
print('Max len targets:',max_len_target)

Max len targets: 19


In [ ]:
encoder_inputs = pad_sequences(input_sequences, maxlen=max_len_input)
print("encoder_inputs.shape:", encoder_inputs.shape)
print("encoder_inputs[0]:", encoder_inputs[0])

encoder_inputs.shape: (4619, 20)
encoder_inputs[0]: [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 27]


In [ ]:
decoder_inputs = pad_sequences(target_sequences_inputs, maxlen=max_len_target, padding='post')
print("decoder_inputs[0]:", decoder_inputs[0])
print("decoder_inputs.shape:", decoder_inputs.shape)

decoder_inputs[0]: [  2 170   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]
decoder_inputs.shape: (4619, 19)


In [ ]:
decoder_targets = pad_sequences(target_sequences, maxlen=max_len_target, padding='post')
print("decoder_targets[0]:", decoder_targets[0])
print("decoder_targets.shape:", decoder_targets.shape)

decoder_targets[0]: [170   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0]
decoder_targets.shape: (4619, 19)


### Loading Glove Vectors

In [ ]:
word2vec = {}
glove_file = open(r'./drive/MyDrive/Speech Conversion//glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
    
glove_file.close()
print('Found {} word vectors'.format(len(word2vec)))

Found 400000 word vectors


### Creating Embedding matrix

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_SIZE))

In [ ]:
for word, i in word2idx_inputs.items():
    if i < MAX_NUM_WORDS:
        
        embedding_vector = word2vec.get(word)
    
        if embedding_vector is not None:
     
          embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(1903, 100)

### Building Model

#### Embedding layer 

In [ ]:
embedding_layer = Embedding(num_words,EMBEDDING_SIZE, weights=[embedding_matrix],input_length=max_len_input,
                           trainable = True)

#### One hot targets

In [ ]:
decoder_targets_one_hot = np.zeros((len(input_texts),max_len_target,num_words_output),dtype=np.float32)

In [ ]:
for i,d in enumerate(decoder_targets):
    
    for t,word in enumerate(d):
        
        decoder_targets_one_hot[i,t,word] = 1

In [ ]:
decoder_targets_one_hot.shape

(4619, 19, 3658)

#### Encoder

In [ ]:
encoder_inputs_placeholder = Input(shape=(max_len_input,))

In [ ]:
x = embedding_layer(encoder_inputs_placeholder)

In [ ]:
encoder = LSTM(LSTM_NODES,return_state=True)

In [ ]:
encoder_outputs,h,c = encoder(x)

In [ ]:
# keep only the states to pass into decoder
encoder_states = [h,c]

#### Decoder

In [ ]:
decoder_inputs_placeholder = Input(shape=(max_len_target,))

In [ ]:
decoder_embedding = Embedding(num_words_output, LSTM_NODES)

In [ ]:
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)

In [ ]:
decoder_lstm = LSTM(LSTM_NODES,return_sequences=True,return_state=True)

In [ ]:
decoder_outputs,_,_ = decoder_lstm(decoder_inputs_x,initial_state=encoder_states)

In [ ]:
decoder_dense = Dense(num_words_output, activation='softmax')

In [ ]:
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs_placeholder, decoder_inputs_placeholder], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
train = model.fit([encoder_inputs, decoder_inputs], decoder_targets_one_hot,batch_size=BATCH_SIZE,verbose=1,
               epochs=EPOCHS,validation_split=0.2,)


Epoch 1/20
58/58 [==============================] - 11s 50ms/step - loss: 1.9833 - accuracy: 0.7565 - val_loss: 2.9200 - val_accuracy: 0.6140
Epoch 2/20
58/58 [==============================] - 2s 31ms/step - loss: 1.3177 - accuracy: 0.8131 - val_loss: 2.7042 - val_accuracy: 0.6616
Epoch 3/20
58/58 [==============================] - 2s 31ms/step - loss: 1.2125 - accuracy: 0.8196 - val_loss: 2.7117 - val_accuracy: 0.6671
Epoch 4/20
58/58 [==============================] - 2s 30ms/step - loss: 1.1246 - accuracy: 0.8238 - val_loss: 2.6822 - val_accuracy: 0.6701
Epoch 5/20
58/58 [==============================] - 2s 30ms/step - loss: 1.0456 - accuracy: 0.8291 - val_loss: 2.6949 - val_accuracy: 0.6734
Epoch 6/20
58/58 [==============================] - 2s 30ms/step - loss: 0.9717 - accuracy: 0.8377 - val_loss: 2.6514 - val_accuracy: 0.6789
Epoch 7/20
58/58 [==============================] - 2s 30ms/step - loss: 0.9034 - accuracy: 0.8455 - val_loss: 2.5887 - val_accuracy: 0.6782
Epoch 8/20
5

#### Setting up the model 

In [ ]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

In [ ]:
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]

In [ ]:
# This is the input to the decoder LSTM that will generate translation one at a time 

decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

In [ ]:
decoder_outputs, h, c =  decoder_lstm(decoder_inputs_single_x,initial_state=decoder_states_inputs)

In [ ]:
decoder_states = [h, c]

In [ ]:
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
decoder_model = Model([decoder_inputs_single] + decoder_states_inputs, [decoder_outputs] + decoder_states)


#### Mapping indexes back to real words 

In [ ]:
idx2word_eng = {v:k for k, v in word2idx_inputs.items()}
idx2word_trans = {v:k for k, v in word2idx_outputs.items()}

#### Sampling Function

In [ ]:
def gen_model (input_seq):
    
    states_value = encoder_model.predict(input_seq)

  
    target_seq = np.zeros((1, 1))

  
    target_seq[0, 0] = word2idx_outputs['<sos>']

  
    eos = word2idx_outputs['<eos>']

 
    output_sentence = []
    
    for _ in range(max_len_target):
        
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # Grab next word
        
        idx = np.argmax(output_tokens.flatten())

        # End sentence of EOS
        if eos == idx:
            break

        word = ''
        if idx > 0:
            word = idx2word_trans[idx]
            output_sentence.append(word)

        # Update the decoder input
        
        target_seq[0, 0] = idx

        # Update states
        
        states_value = [h, c]
        
        
    return ' '.join(output_sentence)


In [ ]:
i = np.random.choice(len(input_texts))

english_input = encoder_inputs[i:i+1]
bangla_translation = gen_model(english_input)

print('-------------')

print('English Input:',input_texts[i])
print('Predicted translation:',bangla_translation)
print('Actual translation:',target_texts[i])


-------------
English Input: I heard you screaming.
Predicted translation: আমি তোমাকে চিৎকার করতে শুনলাম।

Actual translation: আমি তোমাকে চিৎকার করতে শুনলাম।
 <eos>


In [ ]:
input_sen = 'you '
english_input = list()
wordList = list()
# print(word2idx_inputs)

splits = input_sen.split()
n = max_len_input - len(splits)

try:
    for i in range(n):
        wordList.append(0)
    for word in splits:
        wordVal = word2idx_inputs[word]
        wordList.append(wordVal)

    english_input.append(wordList)
    bangla_translation = gen_model(english_input)
    print(bangla_translation)

except:
        translator = Translator()
        res = translator.translate(input_sen,dest='bn')
        print(res)

আপনি কেমন না?

